## 数据预处理
按照中文词组、中文单字、中文拼音方式分词

根据预训练好的word2vec和fasttext词典构建词向量

In [1]:
import sys
stdi, stdo, stde = sys.stdin, sys.stdout, sys.stderr
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdin, sys.stdout, sys.stderr = stdi, stdo, stde
import random
import numpy as np
import pandas as pd
from tflearn.data_utils import pad_sequences
from collections import Counter 
from langconv import *
import os
import csv
import jieba
jieba.add_word('花呗')
jieba.add_word('借呗')
jieba.add_word('收钱码')
jieba.add_word('收款码')

/home/chenchi/anaconda3/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.205 seconds.
Prefix dict has been built succesfully.


In [2]:
train_data = pd.read_csv('../data/aux/train_parse.csv', sep='\t', header=None)#train
test_data = pd.read_csv('../data/test/test.csv', sep='\t', header=None)

In [3]:
train_data.head()

,0,1,2,3
0,1,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1
1,2,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0
2,3,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0
3,4,如何得知关闭借呗,想永久关闭借呗,0
4,5,花呗扫码付钱,二维码扫描可以用花呗吗,0


In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98976 entries, 0 to 98975
Data columns (total 4 columns):
0    98976 non-null int64
1    98976 non-null object
2    98976 non-null object
3    98976 non-null int64
dtypes: int64(2), object(2)
memory usage: 3.0+ MB


In [101]:
# 转换繁体到简体
def cht_to_chs(line):
    line = Converter('zh-hans').convert(line)
    line.encode('utf-8')
    return line
def parse_df(df):
    size = df.shape[0]
    parse_data = df.copy()
    for i in range(size):
        parse_data.iloc[i, 1] = cht_to_chs(df.iloc[i, 1].decode('UTF-8'))
        parse_data.iloc[i, 2] = cht_to_chs(df.iloc[i, 2].decode('UTF-8'))
    return parse_data
train_data = parse_df(train_data)
test_data = parse_df(test_data)
train_data.to_csv('../data/aux/train_parse.csv', sep='\t', header=None, index=False)
print train_data.shape

(98976, 4)


In [9]:
PAD_ID = 0
UNK_ID=1
def translate(text, translation):
    for token, replacement in translation.items():
        text = text.replace(token, ' ' + replacement + ' ')
    text = text.replace('  ', ' ')
    return text
def token_string_as_list(string, token_string='char'):
    string = string.decode('UTF-8')
    translation = {
        '***':'*',
        '花被':'花呗'
    }
    translate(string, translation)
    length = len(string)
    if token_string == 'char':
        listt = [string[i] for i in range(length)]
    elif token_string == 'word':
        listt = jieba.lcut(string)
    elif token_string == 'pinyin':
        string = ''.join(jibe.lcut(string))
        listt = ''.join(lazy_pinyin(string)).split()
    listt = [item for item in listt if item.strip()]
    return listt
def create_voca(train_data, token_string='char', voca_size=5000):
    size = train_data.shape[0]
    input_count = Counter()
    for i in range(size):
        #print train_data.iloc[i, 0], train_data.iloc[i, 1]
        token_list = token_string_as_list(train_data.iloc[i, 1], token_string=token_string)
        input_count.update(token_list)
        token_list = token_string_as_list(train_data.iloc[i, 2], token_string=token_string)
        input_count.update(token_list)
    vocab_worddict = {}
    vocab_indexdict = {}
    vocab_list = input_count.most_common(voca_size)
    for i, tuplee in enumerate(vocab_list):
        word,_ = tuplee
        vocab_worddict[i+2] = word
        vocab_indexdict[word] = i+2
    return vocab_worddict, vocab_indexdict
def parse_train_data(train_data, vocab_indexdict, token_string):
    size = train_data.shape[0]
    input_count = Counter()
    parse_data = train_data.copy()
    temp_data = train_data.copy()
    for i in range(size):
        token_list1 = token_string_as_list(train_data.iloc[i, 1], token_string=token_string)
        token_list2 = token_string_as_list(train_data.iloc[i, 2], token_string=token_string)
        temp_data.iloc[i, 1] = ' '.join(token_list1)
        temp_data.iloc[i, 2] = ' '.join(token_list2)
        parse_data.iloc[i, 1] = ' '.join([str(vocab_indexdict[item]) for item in token_list1 if item in vocab_indexdict])
        parse_data.iloc[i, 2] = ' '.join([str(vocab_indexdict[item]) for item in token_list2 if item in vocab_indexdict])
    if token_string == 'word':
        temp_data.to_csv('../data/aux/train_word.csv', sep='\t', header=None, index=False)
    return parse_data

In [10]:
#vocab_worddict_char, vocab_indexdict_char = create_voca(train_data, token_string='char')
vocab_worddict_word, vocab_indexdict_word = create_voca(train_data, token_string='word')
parse_data2 = parse_train_data(train_data, vocab_indexdict_word,token_string='word')

In [11]:
import json
# jsObj = json.dumps(vocab_indexdict_char)  
# fileObject = open('../data/aux/vocab_indexdict_char.json', 'w')  
# fileObject.write(jsObj)  
# fileObject.close()  
jsObj = json.dumps(vocab_indexdict_word)  
fileObject = open('../data/aux/vocab_indexdict_word.json', 'w')  
fileObject.write(jsObj)  
fileObject.close()  

In [12]:
#parse_data1 = parse_train_data(train_data, vocab_indexdict_char,token_string='char')
parse_data2 = parse_train_data(train_data, vocab_indexdict_word,token_string='word')
#parse_data1.to_csv('../data/aux/train_char_indexvec.csv', index=False)
parse_data2.to_csv('../data/aux/train_word_indexvec.csv', index=False)

构建词向量

In [6]:
import json
def load_word_vec(file_path):
    source_object = open(file_path, 'r')
    word_vec_dict={}
    for i,line in enumerate(source_object):
        if i==0 and 'word2vec' in file_path:
            continue
        line=line.strip()
        line_list=line.split()
        word=line_list[0].decode("utf-8")
        vec_list=[float(x) for x in line_list[1:]]
        word_vec_dict[word]=np.array(vec_list)
    #print("word_vec_dict:",word_vec_dict)
    return word_vec_dict

In [9]:
with open('../data/aux/vocab_indexdict_char.json') as f:
    vocab_worddict_char = json.load(f)
with open('../data/aux/vocab_indexdict_word.json') as f:
    vocab_worddict_word = json.load(f)


word_vec_fasttext_dict=load_word_vec('../data/aux/fasttext.vec') #word embedding from fasttxt
word_vec_word2vec_dict = load_word_vec('../data/aux/word2vec.txt') #word embedding from word2vec
word_char_dict = load_word_vec('../data/word-character')

In [41]:
print len(vocab_worddict_char)
print len(vocab_worddict_word)

2085
5000


In [10]:
len(word_char_dict)

636037

In [47]:
embedding_matrix = np.zeros((len(vocab_worddict_word) + 2, 300))
for word, index in vocab_worddict_word.items():
    embedding_vector = word_vec_fasttext_dict.get(word)
    #import pdb;pdb.set_trace()
    if embedding_vector is not None:
        embedding_matrix[index][:50] = embedding_vector
    elif word in word_char_dict:
        embedding_matrix[index] = word_char_dict.get(word)

np.save("../data/aux/vec_word.npy",embedding_matrix)

embedding_matrix = np.zeros((len(vocab_worddict_char) + 2, 300))
for word, index in vocab_worddict_char.items():
    if word in word_char_dict:
        embedding_matrix[index] = word_char_dict.get(word)

np.save("../data/aux/vec_char.npy",embedding_matrix)

In [46]:
embedding_matrix[3]

array([ 0.094878, -0.274933,  0.274207, -0.173418,  0.224209, -0.044867,
        0.250689, -0.012698, -0.075608,  0.079465,  0.194921,  0.12656 ,
       -0.118317,  0.356042,  0.191383,  0.077959, -0.334946,  0.212342,
        0.07326 ,  0.254857, -0.077326, -0.016247,  0.249023,  0.253988,
        0.145813,  0.310039, -0.22933 , -0.195232, -0.207432,  0.04946 ,
        0.444571,  0.053893, -0.276234,  0.119278,  0.123157, -0.154975,
        0.159252, -0.233318,  0.041756,  0.096739,  0.125562, -0.132592,
        0.085903,  0.127125,  0.079671, -0.026953, -0.028784, -0.138333,
       -0.06711 ,  0.119755,  0.046799, -0.043481,  0.240149,  0.068734,
        0.105269, -0.009649, -0.056001, -0.058492,  0.184133,  0.175659,
        0.105764, -0.158145,  0.140379,  0.111549,  0.043281,  0.251387,
       -0.36871 ,  0.198847, -0.27882 , -0.132679,  0.119707,  0.019584,
        0.00324 ,  0.012386, -0.111798,  0.121082, -0.044826, -0.02054 ,
        0.0914  ,  0.009025,  0.028984, -0.182722, 

In [83]:
word_vec_fasttext_dict[u'花呗'].shape

(50,)

In [39]:
count = 0
for key in vocab_worddict_char:
    #
    if key not in word_char_dict:
        #if key in word_vec_fasttext_dict:
        print key
        count += 1

﻿
玏
睌
叧
徣
嚒
呮
挷
幵
牫
鈤
肔


In [17]:
count

1267

In [79]:
len(vocab_worddict_word)

5000

In [ ]:
len(vocab_worddict_word)

In [26]:
u'借呗' in word_vec_fasttext_dict

True

In [30]:
word_vec_fasttext_dict[u'借呗'].shape

(50,)

In [29]:
word_char_dict[u'银行'].shape

(300,)

In [36]:
a = np.zeros((300, ))

In [37]:
a[:50] = word_vec_fasttext_dict[u'借呗']

In [38]:
a

array([-0.27397 ,  0.64121 , -0.22201 ,  0.31229 ,  0.62923 ,  0.17857 ,
        0.23869 ,  0.96237 ,  0.23135 ,  0.13782 , -0.034528,  0.18526 ,
       -0.33951 , -0.91978 , -0.16099 , -0.34089 , -0.58058 ,  0.52549 ,
        0.81099 ,  0.37196 , -0.094091, -0.24108 , -0.43252 , -0.61711 ,
       -0.1377  ,  0.039142,  0.10998 , -1.0043  , -0.22509 ,  1.5202  ,
       -0.17654 ,  0.63722 ,  0.31804 ,  0.29838 ,  0.038973, -0.36773 ,
       -0.070742,  0.060467, -0.24382 , -0.42763 , -0.072725,  0.066941,
       -0.12018 ,  0.51183 ,  1.8377  ,  0.3385  , -0.08096 , -0.24191 ,
       -0.10926 ,  0.038589,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      , 